## Introduzione

In questo corso condenseremo le lezioni provenienti dai seguenti 4 corsi

* [3Blue 1Brown Essence of Linear Algebra](https://www.youtube.com/watch?v=fNk_zzaMoSs&list=PLZHQObOWTQDPD3MizzM2xVFitgF8hE_ab&index=1&t=1s&ab_channel=3Blue1Brown)
* [Introduction to Linear Algebra](https://www.udacity.com/course/linear-algebra-refresher-course--ud953)
* [Eigenvectors and eigenvalues](https://www.udacity.com/course/eigenvectors-and-eigenvalues--ud104)
* [Computational linear algebra](https://www.youtube.com/playlist?list=PLtmWHNX-gukIc92m1K0P6bIOnZb-mg0hY)


partiamo subito parlando dei due oggetti fondamentali dell'algebra, i punti ed i vettori.
Un punto rappresenta una locazione nello spazio e viene scritto anche come $(x,y,z,...)$.

Per esempio in un sistema cartesiano (due assi x e y) possiamo rappresentare il punto (2,-1) come un punto nel seguente modo:

![punto](images\point.png)

Possiamo rappresentare il punto con un sistema ordinato di coordinate dove nella prima posizione si mette la coordinata x e nella seconda posizione la coordinata y

Un vettore rappresenta un cambio di locazione, puo essere rappresentato come una freccia che unisce due punti.
Due proprietà importanti che rappresentato un vettore sono la magnitudine (o modulo) e la sua direzione 

![vettore](images\vector.png)

ad esempio il vettore sottostante rappresenta lo spostamento tra il punto (2,-1) e (4,2) possiamo rappresentare il vettore con la seguente scritta:

$\begin{bmatrix} 2 \\ 3 \end{bmatrix}$

La principale differenza tra un vettore e un punto è che un vettore non ha un punto fisso, possiamo creare una freccia uguale partendo da due punti differenti ottenendo lo stesso risultato

![vettori](images\vectors.png)

Da qui una considerazione, due vettori sono uguali se rappresentano la stesso spostamento nella stessa direzione, mentre due punti sono uguali se rappresentano lo stesso posto.

Nonostante questa differenza, quando studiamo l'Algebra Lineare di solito confondiamo la nozione di punto e di vettore.

## Operazioni sui vettori

Comunciamo a creare una piccola libreria che utilizzeremo in tutto il corso per capire il funzionamento dei vettori.
Questa è implementata al solo scopo educativo, se avessimo necessità di fare queste operazioni si consiglia sl'utilizzo della libreria numpy

Creiamo subito la classe Vector e implementiamo due funzioni base:

* __str__ che crea una rappresentazione a stringa dei valori del vettore
* __eq__ che controlla se due vettori sono uguali

In [1]:
class Vector(object):
    def __init__(self, coordinates):
        try:
            if not coordinates:
                raise ValueError
            self.coordinates = tuple(coordinates)
            self.dimension = len(coordinates)

        except ValueError:
            raise ValueError('The coordinates must be nonempty')

        except TypeError:
            raise TypeError('The coordinates must be an iterable')


    def __str__(self):
        return 'Vector: {}'.format(self.coordinates)


    def __eq__(self, v):
        return self.coordinates == v.coordinates

In [2]:
v1 = Vector([1,2,3])
print(v1)

Vector: (1, 2, 3)


In [3]:
v2 =  Vector([-1,2,3])
v3 =  Vector([1,2,3])

v1 == v2

False

In [4]:
v1 == v3

True

aggiungiamo ora ulteriori operazioni alla nostra classe.
L'operazione di somma e sottrazione lavorano elemento per elemento i valori del vettore, mentre la moltiplicazione scalare moltiplica il valore su tutti gli elementi del vettore

In [5]:
from decimal import Decimal

class Vector(object):
    def __init__(self, coordinates):
        try:
            if not coordinates:
                raise ValueError
            self.coordinates = tuple([Decimal(c) for c in coordinates])
            self.dimension = len(coordinates)

        except ValueError:
            raise ValueError('The coordinates must be nonempty')

        except TypeError:
            raise TypeError('The coordinates must be an iterable')
        
    def __len__(self):
        return len(self.coordinates)

    def __str__(self):
        return 'Vector: {}'.format([round(coord, 3)
                                    for coord in self.coordinates])

    def __eq__(self, v):
        return self.coordinates == v.coordinates
    
    def __add__(self, other):
        return self.plus(other)

    def plus(self, other):
        return Vector([x + y for x, y in zip(self.coordinates, other.coordinates)])
    
    def __sub__(self, other):
        return self.minus(other)

    def minus(self, other):
        return Vector([x - y for x, y in zip(self.coordinates, other.coordinates)])

    def times_scalar(self, factor):
        return Vector([Decimal(factor) * coord for coord in self.coordinates])

In [6]:
v2 =  Vector([-1,2,3])
v3 =  Vector([1,2,3])

print(v2 - v3)

Vector: [Decimal('-2.000'), Decimal('0.000'), Decimal('0.000')]


In [7]:
print(v2.times_scalar(2))

Vector: [Decimal('-2.000'), Decimal('4.000'), Decimal('6.000')]


## Ampiezza e direzione

Aumentiamo le operazioni che possono essere fatte tra vettori, l'ampiezza e la direzione.
Per calcolare l'ampiezza di un vettore usiamo la consueta formula di Pitagora.
In $n$ dimensioni la formula diventa:

$$\begin{bmatrix} v_1 \\ v_2 \\ \vdots \\ v_n \end{bmatrix}$$


$$\lVert \vec{v} \rVert = \sqrt{v_1^2 + v_2^2 + \dots + v_n^2}$$

detto questo diciamo che il vettore unitario è rappresentato da un vettore di lunghezza unitaria, ma questo rappresenta anche la sua direzione.

L'operazione di calcolare un vettore unitario partendo dal nostro vettore è anche chiamata normalizzazione, il primo passo sta nel calcolare la sua ampiezza (mangitudine) per poi eseguire una moltiplicazione scalare vista prima nel seguente modo.


$$\frac{1}{\lVert \vec{v} \rVert} \cdot  \vec{v}$$


Facciamo notare una cosa, prendiamo il vettore con tutti elementi 0:

$$\begin{bmatrix} 0 \\ 0 \\ \vdots \\ 0 \end{bmatrix} = \vec{0}$$

Questo vettore indica che non c'è stato nessuno spostamento perciò 

$$\lVert \vec{0} \rVert = 0$$

Il problema è che la formula 

$$\frac{1}{\lVert \vec{0} \rVert} \cdot  \vec{0}$$

Da problemi con la divisione per 0.
Il vettore $\vec{0}$ non ha normalizzazione, possiamo esprimere questo dicendo che il vettore $\vec{0}$ non ha direzione.

Passiamo alla implementazione di queste operazioni

In [8]:
from math import sqrt
from decimal import Decimal


class Vector(object):
    def __init__(self, coordinates):
        try:
            if not coordinates:
                raise ValueError
            self.coordinates = tuple([Decimal(c) for c in coordinates])
            self.dimension = len(coordinates)

        except ValueError:
            raise ValueError('The coordinates must be nonempty')

        except TypeError:
            raise TypeError('The coordinates must be an iterable')

    
    def __len__(self):
        return len(self.coordinates)

    def __getitem__(self, i):
        return self.coordinates[i]

    def __str__(self):
        return 'Vector: {}'.format([round(coord, 3)
                                    for coord in self.coordinates])

    def __eq__(self, v):
        return self.coordinates == v.coordinates

    def is_zero(self):
        return set(self.coordinates) == set([Decimal(0)])
    
    def __add__(self, other):
        return self.plus(other)

    def plus(self, other):
        return Vector([x + y for x, y in zip(self.coordinates, other.coordinates)])
    
    def __sub__(self, other):
        return self.minus(other)

    def minus(self, other):
        return Vector([x - y for x, y in zip(self.coordinates, other.coordinates)])

    def times_scalar(self, factor):
        return Vector([Decimal(factor) * coord for coord in self.coordinates])
    

    def magnitude(self):
        return Decimal(sqrt(sum([coord * coord for coord in self.coordinates])))

    def normalize(self):
        try:
            return self.times_scalar(Decimal('1.0') / self.magnitude())
        except ZeroDivisionError:
            raise Exception('Cannot normalize the zero vector')


In [9]:
v1 =  Vector([1,2,3])

print("{0:.2f}".format(v1.magnitude()))
print(v1.normalize())

3.74
Vector: [Decimal('0.267'), Decimal('0.535'), Decimal('0.802')]


In [10]:
zero =  Vector([0,0,0])
try:
    zero.normalize()
except Exception as e:
    print(e)

Cannot normalize the zero vector


## Combinazioni lineari, Copertura e base

Introciamo un concetto che viene sviscerato in questa lezione:

[![Base](https://img.youtube.com/vi/k7RM-ot2NWY/0.jpg)](https://www.youtube.com/watch?v=k7RM-ot2NWY)

prendiamo due vettori $\vec{u}$ e $\vec{v}$ l'insieme di tutte le combinazioni lineari possibili:

$$a\vec{v} + b\vec{u}$$

viene chiamato copertura, diciamo un vettore $\vec{w}$ è linearmente dipendente dai vettori $\vec{u}$ e $\vec{v}$ se esiste una combinazione lineare di valori di $a$ e $b$ che genera $\vec{w}$

## Prodotto scalare

Abbiamo appena visto come sommare i vettori e come si esegue la moltiplicazione con una costante, possiamo però chiederci come funziona la moltiplicazione tra vettori.

Scopriremo che esistono molte versioni di moltiplicazione tra vettori, prendere un elemento di un vettore e moltiplicarlo con il suo corrispettivo del secondo non ci porta a risultati interessanti.

Ci sono altre definizioni di moltiplicazione che sono più utili ai nostri scopi.
La prima nozione di prodotto che analizzeremo è il prodotto scalare che di solito viene scritta nel seguente modo:

$$\vec{u} \cdot \vec{v}$$

questa moltiplicazione è molto importante in quanto ci permette di calcolare l'angolo che intercorre tra due vettori.
Più precisamente possiamo definire 

$$\vec{u} \cdot \vec{w} = \lVert \vec{u} \rVert \cdot \lVert \vec{w} \rVert \cdot \cos \theta$$

Dunque il prodotto scalare rappresenta il prodotto tra il modulo dei due vettori moltiplicato il coseno dell'angolo che intercorre tra i due.

![product](images\dot_product.png)


Il risultato di questa operazione è un numero non un vettore come si può intuire inoltre con questa operazione possiam calcolare l'angolo che intercorre tra i due vettori mediante questa operazione:

$$\theta = \arccos{\left(\frac{\vec{u} \cdot \vec{w}}{\lVert \vec{u} \rVert \cdot \lVert \vec{w} \rVert}\right)}$$

Possiamo vedere questa formula anche come l'arcocoseno del prodotto scalare dei vettori normalizzati $\vec{u}$ e $\vec{w}$

$$\theta = \arccos{\left(\frac{1}{\lVert \vec{u} \rVert} \vec{u} \cdot \frac{1}{\lVert \vec{w} \rVert} \vec{w}\right)}$$

Una formula più leggibile è espressa nel seguente modo:

$$\vec{v} \cdot \vec{w} = v_1 \cdot w_1 + v_2 \cdot w_2 + \cdots + v_n \cdot w_n$$

Ora sappiamo dalla geometria che il coseno assume valori tra 1 e -1 possiamo dunque riscrivere la formula sopra come:


$$-\lVert \vec{u} \rVert \cdot \lVert \vec{w} \rVert \leq \vec{u} \cdot \vec{w} \leq \lVert \vec{u} \rVert \cdot \lVert \vec{w} \rVert$$

Possiamo riscrivere la disequazione nel seguente modo:

$$\lvert \vec{u} \cdot \vec{w} \rvert \leq \lVert \vec{u} \rVert \cdot \lVert \vec{w} \rVert$$

questa viene anche chiamata disuguaglianza di Cauchy-Schwarz.
Se i due vettori sono paralleli avremo che 

$$\vec{u} \cdot \vec{w} = \lVert \vec{u} \rVert \cdot \lVert \vec{w} \rVert$$

Similmente se i due vettori sono opposti avremo che 

$$\vec{u} \cdot \vec{w} = - \lVert \vec{u} \rVert \cdot \lVert \vec{w} \rVert$$

Mentre se i vettori sono ortogonali avremo che 

$$\vec{u} \cdot \vec{w} = 0$$

Inoltre possiamo vedere facilmente che 

$$\vec{v} \cdot \vec{v} = \lVert \vec{v} \rVert ^2$$

Perciò

$$\lVert \vec{v} \rVert = \sqrt{\vec{v} \cdot \vec{v}}$$

qui la spiegazione grafica

[![prodotto scalare](https://img.youtube.com/vi/LyGKycYT2v0/0.jpg)](https://www.youtube.com/watch?v=LyGKycYT2v0)



In [11]:
from math import acos, sqrt, pi
from decimal import Decimal


class Vector(object):
    def __init__(self, coordinates):
        try:
            if not coordinates:
                raise ValueError
            self.coordinates = tuple([Decimal(c) for c in coordinates])
            self.dimension = len(coordinates)

        except ValueError:
            raise ValueError('The coordinates must be nonempty')

        except TypeError:
            raise TypeError('The coordinates must be an iterable')

    
    def __len__(self):
        return len(self.coordinates)

    def __getitem__(self, i):
        return self.coordinates[i]

    def __str__(self):
        return 'Vector: {}'.format([round(coord, 3) for coord in self.coordinates])

    def __eq__(self, v):
        return self.coordinates == v.coordinates

    def is_zero(self):
        return set(self.coordinates) == set([Decimal(0)])
    
    def __add__(self, other):
        return self.plus(other)

    def plus(self, other):
        return Vector([x + y for x, y in zip(self.coordinates, other.coordinates)])
    
    def __sub__(self, other):
        return self.minus(other)

    def minus(self, other):
        return Vector([x - y for x, y in zip(self.coordinates, other.coordinates)])

    def times_scalar(self, factor):
        return Vector([Decimal(factor) * coord for coord in self.coordinates])
    

    def magnitude(self):
        return Decimal(sqrt(sum([coord * coord for coord in self.coordinates])))

    def normalize(self):
        try:
            return self.times_scalar(Decimal('1.0') / self.magnitude())
        except ZeroDivisionError:
            raise Exception('Cannot normalize the zero vector')


    def dot_product(self, other):
        return sum(x * y for x, y in zip(self.coordinates, other.coordinates))

    def get_angle_rad(self, other):
        dot_prod = round(self.normalize().dot_product(other.normalize()), 3)
        return acos(dot_prod)

    def get_angle_deg(self, other):
        degrees_per_rad = 180. / pi
        return degrees_per_rad * self.get_angle_rad(other)

    def is_parallel(self, other):
        return (self.is_zero() or other.is_zero() or
                self.get_angle_rad(other) in [0, pi])

    def is_orthogonal(self, other):
        return round(self.dot_product(other), 3) == 0


In [12]:
v1 = Vector([1,2,3])
v2 = Vector([4,5,6])
v1.dot_product(v2)

Decimal('32')

## Proiezione dei vettori

Parliamo di una nuova operazione che possiamo implementare tra due vettori, la proiezione.
Prendiamo il vettore $\vec{v}$ e il vettore $\vec{b}$ se eseguiamo la proiezione di $\vec{v}$ su $\vec{b}$ che chiameremo $\vec{v^\lVert}$ supponendo che l'angolo tra i due sia minore o uguale di 90°

![projection](images\projection.png)

$$\lVert \vec{v^\lVert} \rVert= \lVert \vec{v} \rVert  \cdot \cos\theta$$


Se esegiamo la sostituzione di $\cos\theta$ otteniamo la nuova formula:

$$\lVert \vec{v^\lVert} \rVert = \lVert \vec{v} \rVert  \cdot \left(\frac{\vec{v} \cdot \vec{b}}{\lVert \vec{v} \rVert \cdot \lVert \vec{b} \rVert}\right) = \vec{v} \cdot \vec{u_b}$$

La scrittura $\vec{u_b}$ indica il vettore normalizzato b.
Ora visto che $\vec{u_b}$ è un vettore unitario e che i due vettori $\vec{u_b}$ e $\vec{v}$ puntano nella stessa direzione possiamo anche scrivere che:

$\lVert \vec{v^\lVert} \rVert \cdot \vec{u_b} =  \vec{v^\lVert}$

Unendo le due formule abbiamo che:

$$\vec{v^\lVert} = \left(\vec{v} \cdot \vec{u_b}\right) \cdot \vec{u_b}$$


Ora se l'angolo supera i 90° la formula diventa:

$$\lVert \vec{v^\lVert} \rVert =  -\vec{v} \cdot \vec{u_b}$$ 


Il vettore $\vec{v^\lVert}$ ora punta nella direzione opposta a $\vec{b}$, perciò possiamo riscrivere la formula come:

$-\lVert \vec{v^\lVert} \rVert \cdot \vec{u_b} =  \vec{v^\lVert}$

E unendo le due formule possiamo annullare il segno meno ottenendo di nuovo:

$$\vec{v^\lVert} = \left(\vec{v} \cdot \vec{u_b}\right) \cdot \vec{u_b}$$

implementiamo anche questa operazione:

In [13]:
from math import acos, sqrt, pi
from decimal import Decimal


class Vector(object):
    def __init__(self, coordinates):
        try:
            if not coordinates:
                raise ValueError
            self.coordinates = tuple([Decimal(c) for c in coordinates])
            self.dimension = len(coordinates)

        except ValueError:
            raise ValueError('The coordinates must be nonempty')

        except TypeError:
            raise TypeError('The coordinates must be an iterable')

    
    def __len__(self):
        return len(self.coordinates)

    def __getitem__(self, i):
        return self.coordinates[i]

    def __str__(self):
        return 'Vector: {}'.format([round(coord, 3) for coord in self.coordinates])

    def __eq__(self, v):
        return self.coordinates == v.coordinates

    def is_zero(self):
        return set(self.coordinates) == set([Decimal(0)])
    
    def __add__(self, other):
        return self.plus(other)

    def plus(self, other):
        return Vector([x + y for x, y in zip(self.coordinates, other.coordinates)])
    
    def __sub__(self, other):
        return self.minus(other)

    def minus(self, other):
        return Vector([x - y for x, y in zip(self.coordinates, other.coordinates)])

    def times_scalar(self, factor):
        return Vector([Decimal(factor) * coord for coord in self.coordinates])
    

    def magnitude(self):
        return Decimal(sqrt(sum([coord * coord for coord in self.coordinates])))

    def normalize(self):
        try:
            return self.times_scalar(Decimal('1.0') / self.magnitude())
        except ZeroDivisionError:
            raise Exception('Cannot normalize the zero vector')


    def dot_product(self, other):
        return sum(x * y for x, y in zip(self.coordinates, other.coordinates))

    def get_angle_rad(self, other):
        dot_prod = round(self.normalize().dot_product(other.normalize()), 3)
        return acos(dot_prod)

    def get_angle_deg(self, other):
        degrees_per_rad = 180. / pi
        return degrees_per_rad * self.get_angle_rad(other)

    def is_parallel(self, other):
        return (self.is_zero() or other.is_zero() or
                self.get_angle_rad(other) in [0, pi])

    def is_orthogonal(self, other):
        return round(self.dot_product(other), 3) == 0
        
    def get_projected_vector(self, other):
        b_normalized = other.normalize()
        return b_normalized.times_scalar(self.dot_product(b_normalized))


In [14]:
v1 = Vector([1,2,3])
v2 = Vector([4,5,6])
print(v1.get_projected_vector(v2))

Vector: [Decimal('1.662'), Decimal('2.078'), Decimal('2.494')]


## Prodotto vettoriale

Qui abbiamo un altra nozione di moltiplicazioni tra due vettori, che esiste soltanto in uno spazio tridimensionale.
Geometricamente il prodotto vettoriale di due vettori $\vec{v}$ e $\vec{w}$ è un vettore:

* Ortogonale a $\vec{v}$ e $\vec{w}$
* e rispetta la seguente ugualianza $\lVert \vec{v} \times \vec{w} \rVert = \lVert \vec{v} \rVert \lVert \vec{w} \rVert sin \theta$

Ora usando solo queste definizioni sappiamo che se $\theta$ è uguale a $0$ o $\pi$ (0,180) $\lVert \vec{v} \times \vec{w} \rVert = 0$ e se o $\vec{v} = 0$ o $\vec{w} = 0$ avremo che $\vec{v} \times \vec{w} = 0$

![cross](images\cross_product.png)


Ora usando le definizioni che abbiamo visto sopra dobbiamo decidere il verso

![cross](images\cross_product_verso.png)

Per decidere il verso si usa la regola della mano destra. Si usa il pollice per seguire la direzione di $\vec{v}$ l'indice segue la direzione di $\vec{w}$ il medio deve essere perpendicolare alle due dita ci darà il verso del risultato.

![mano](images\regola-mano-destra.png)

Da questa regola deriviamo che il prodotto vettoriale è anticommutativo cioè:

$$\vec{v} \times \vec{w} = -(\vec{v} \times \vec{w})$$

La formula per il calcolo $$\vec{v} \times \vec{w}$$ può essere espressa nel seguente modo se vediamo $\vec{v}$ nel seguente modo:

$$\vec{v} = \begin{bmatrix} x_1 \\ y_1 \\ z_1 \end{bmatrix}$$

e $\vec{w}$ nel modo seguente:

$$\vec{w} = \begin{bmatrix} x_2 \\ y_2 \\ z_2 \end{bmatrix}$$

definiamo il risultato:

$$\vec{v} \times \vec{w} = \begin{bmatrix} y_1 z_2 - y_2 z_1\\ -(x_1 z_2 - x_2 z_1) \\ x_1 y_2 - x_2 y_1 \end{bmatrix}$$


[![prodotto vettoriale](https://img.youtube.com/vi/eu6i7WJeinw/0.jpg)](https://www.youtube.com/watch?v=eu6i7WJeinw)


In [19]:
from math import acos, sqrt, pi
from decimal import Decimal


class Vector(object):
    def __init__(self, coordinates):
        try:
            if not coordinates:
                raise ValueError
            self.coordinates = tuple([Decimal(c) for c in coordinates])
            self.dimension = len(coordinates)

        except ValueError:
            raise ValueError('The coordinates must be nonempty')

        except TypeError:
            raise TypeError('The coordinates must be an iterable')

    
    def __len__(self):
        return len(self.coordinates)

    def __getitem__(self, i):
        return self.coordinates[i]

    def __str__(self):
        return 'Vector: {}'.format([round(coord, 3) for coord in self.coordinates])

    def __eq__(self, v):
        return self.coordinates == v.coordinates

    def is_zero(self):
        return set(self.coordinates) == set([Decimal(0)])
    
    def __add__(self, other):
        return self.plus(other)

    def plus(self, other):
        return Vector([x + y for x, y in zip(self.coordinates, other.coordinates)])
    
    def __sub__(self, other):
        return self.minus(other)

    def minus(self, other):
        return Vector([x - y for x, y in zip(self.coordinates, other.coordinates)])

    def times_scalar(self, factor):
        return Vector([Decimal(factor) * coord for coord in self.coordinates])
    

    def magnitude(self):
        return Decimal(sqrt(sum([coord * coord for coord in self.coordinates])))

    def normalize(self):
        try:
            return self.times_scalar(Decimal('1.0') / self.magnitude())
        except ZeroDivisionError:
            raise Exception('Cannot normalize the zero vector')


    def dot_product(self, other):
        return sum(x * y for x, y in zip(self.coordinates, other.coordinates))

    def get_angle_rad(self, other):
        dot_prod = round(self.normalize().dot_product(other.normalize()), 3)
        return acos(dot_prod)

    def get_angle_deg(self, other):
        degrees_per_rad = 180. / pi
        return degrees_per_rad * self.get_angle_rad(other)

    def is_parallel(self, other):
        return (self.is_zero() or other.is_zero() or
                self.get_angle_rad(other) in [0, pi])

    def is_orthogonal(self, other):
        return round(self.dot_product(other), 3) == 0
        
    def get_projected_vector(self, other):
        b_normalized = other.normalize()
        return b_normalized.times_scalar(self.dot_product(b_normalized))

    def cross_product(self, other):
        [x1, y1, z1] = self.coordinates
        [x2, y2, z2] = other.coordinates
        x = (y1 * z2) - (y2 * z1)
        y = -((x1 * z2) - (x2 * z1))
        z = (x1 * y2) - (x2 * y1)
        return Vector([x, y, z])


In [20]:
v1 = Vector([1,2,3])
v2 = Vector([4,5,6])
print(v1.cross_product(v2))

Vector: [Decimal('-3.000'), Decimal('6.000'), Decimal('-3.000')]
